In [5]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-tf-deepfm

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded

Step 1/10 : FROM tensorflow/tensorflow:1.10.1-gpu-py3
 ---> 6243acd2b19f
Step 2/10 : WORKDIR /home/tf/
 ---> Using cache
 ---> 62f9416a5cc0
Step 3/10 : COPY src/ /home/tf/
 ---> 4aa204494886
Step 4/10 : COPY input/ /opt/ml/input/
 ---> 868c402eaf2c
Step 5/10 : RUN mkdir -p /opt/ml/output/
 ---> Running in 1290cde95f0d
Removing intermediate container 1290cde95f0d
 ---> 7566a276fbb8
Step 6/10 : RUN mkdir -p /opt/ml/model/
 ---> Running in bf179e90adb1
Removing intermediate container bf179e90adb1
 ---> 36456c84bb86
Step 7/10 : RUN pip install keras sklearn pydot
 ---> Running in 9560b76886b9
  Running setup.py bdist_wheel for pyyaml: started
  Running setup.py bdist_wheel for pyyaml: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/ad/da/0c/74eb680767247273e2cf2723482cb9c924fe70af57c334513f
Successfully built pyyaml
You are using pip version 18.0, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pi

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

